<a href="https://colab.research.google.com/github/jfjoung/AI_For_Chemistry/blob/main/notebooks/week5/Week_5_palladium_dimers_discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


🎯 학습 목표:

- 화학 구조 데이터를 벡터화하고, 비지도 학습 기법을 활용하여 분자 간 유사성을 분석하는 방법을 익힌다.
- Tanimoto 유사도를 기반으로 한 거리 계산 방식을 이해하고, 이를 클러스터링에 적용하는 과정을 학습한다.
- UMAP(Uniform Manifold Approximation and Projection)을 활용하여 고차원 화학 데이터를 2차원 공간에 시각화하는 방법을 배운다.
- 계층적 군집화(hierarchical clustering)를 적용하여 구조적으로 유사한 팔라듐 이합체(palladium dimers)를 그룹화하는 방법을 익힌다.
- 시각적 도구(예: scatter plot, heatmap 등)를 통해 군집화 결과를 해석하고, 데이터 속에서 유의미한 패턴을 도출하는 능력을 기른다.
- 비지도 학습을 통해 알려지지 않은 화학적 구조나 반응 경향성을 탐색하는 실전 응용 능력을 배양한다.


# Accelerating palladium catalyst discovery: a practical application of unsupervised learning

이 노트북에서는 간단한 알고리즘인 k-means이 어떻게 적은 수의 실험 데이터를 이용해 새로운 촉매를 발견하는 데 활용될 수 있는지를 살펴봅니다. 머신러닝 기법을 활용하면 직관적으로는 예상하기 어려운 패턴을 드러내고, 일반적으로는 간과될 수 있는 실험을 제안할 수 있다는 점을 확인할 수 있습니다.

이 노트북은 Hueffel 외 연구진의 연구를 바탕으로 작성되었으며, 해당 연구는 2021년 *Science* 저널에 게재되었습니다. [Science 논문 링크](https://www.science.org/doi/full/10.1126/science.abj0999)



# 0. 관련 패키지

일반적으로 사용하는 패키지들을 설치하고, 필요한 데이터셋을 다운로드합니다.


In [ ]:
! pip install scikit-learn pandas numpy matplotlib seaborn scipy tqdm

! mkdir data
! wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week5/LKB_P.csv -O data/LKB_P.csv
! wget https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/data/week5/Dimer_LKB_P_temp.csv -O data/Dimer_LKB_P.csv

# 1. 문제 소개

다양한 물리적, 화학적 형태로 존재하는 금속의 다양한 분포, 즉 Metal speciation은 균일계 촉매 반응(homogeneous catalysis)에서 매우 중요한 특성입니다. 이 특성은 촉매의 반응성이나 선택성과 같은 성질을 결정하는 데 핵심적인 역할을 하며, 리간드(ligand)는 metal speciation에 영향을 미치는 주요 요소 중 하나입니다. 일반적으로 구조와 반응성 사이에는 상관관계가 있다고 알려져 있지만, 실제로 리간드가 촉매 활성을 어떻게 조절하는지는 명확히 이해되지 않아 새로운 촉매 설계를 방해하고 있습니다.

<p align="left">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/speciation.png" width="400"/>
</p>

팔라듐(Pd)이 촉매로 작용하는 반응은 균일계 촉매 시스템 중 대표적인 예입니다. 대부분의 촉매 사이클은 Pd(0) 또는 Pd(II) 종(species)을 포함하지만, [Pd(I)(μ-X)P(t-Bu)<sub>3</sub>]<sub>2</sub> 구조를 가진 Pd(I) 이합체는 뛰어난 안정성과 촉매 성능으로 인해 특별히 주목받고 있습니다. 하지만 특정 리간드는 Pd(I) 이합체 생성을 촉진하는 반면, 다른 리간드는 그렇지 않은 이유가 명확히 밝혀지지 않아, 지금까지 이런 화합물은 소수만 알려져 있습니다.

<p align="left">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/pd.png" width="400"/>
</p>

새로운 Pd(I) 종을 발견하는 일반적인 전략은 시행착오(trial and error) 방식입니다. 이 방법으로 Pd(I) 상태를 유도하는 리간드는 단 4종만 발견되었습니다. 이를 극복하기 위한 다양한 시도가 있었으나, 대부분 한계를 가졌습니다. 예를 들어, 348개의 phosphine 리간드에 대해 28개의 일반적인 특성을 기반으로 PCA를 수행했지만, 리간드와 금속 분포 사이의 관계를 밝혀내는 데 실패했습니다. 또한, DFT 계산을 통해 Pd(0)와 Pd(II)가 Pd(I)로 되는 공존 반응의 자유에너지를 예측하는 시도도 있었으나, PdCy<sub>3</sub> (Pd(I)를 유도하지 않는 리간드)의 에너지가 Pd(I)를 유도하는 알려진 리간드들과 거의 동일하게 나타나 이 가설 역시 무효화되었습니다.

<p align="left">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/PdDFT.png" width="400"/>
</p>

이처럼 실험적으로 얻은 데이터가 매우 적은 상황에서, **비지도 학습(unsupervised learning)을 통해 기존의 지식을 확장하고, Pd(I) 이합체를 형성할 수 있는 새로운 리간드 후보를 제안할 수는 없을까요?**


## 연습 1

첫 번째 연습에서는 348개의 phosphine 리간드와 그에 대한 일반적인 descriptor를 포함한 데이터프레임을 불러옵니다. 각 분자는 리간드의 유형을 나타내는 8가지 class 중 하나로 분류되어 있으며, 데이터의 주요 구성 요소 중 첫 번째부터 네 번째까지의 주성분(PCA 결과)도 포함되어 있습니다. 이 연습에서는 PCA가 데이터에 대해 충분한 통찰을 제공하지 못한다는 점을 확인해보겠습니다.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load ligand information
ligands = pd.read_csv('data/LKB_P.csv', sep=';')

ligands.head()

이제 주성분 1 (PC1)과 주성분 2 (PC2)를 사용하여 리간드들을 시각화하고, 각 리간드의 유형은 'Type' 열의 값을 이용하여 표시합니다.


In [ ]:
import seaborn as sns

# seaborn 스타일 적용
sns.set(style="whitegrid")

# PC1과 PC2를 기준으로 산점도 시각화, 'Type' 컬럼으로 색상 구분
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=ligands,             # 데이터프레임 사용
    x="PC1",             # x축: 첫 번째 주성분
    y="PC2",             # y축: 두 번째 주성분
    hue="Type",          # 색상: 리간드 유형
    palette="Set2",      # 색상 팔레트
    s=70                 # 점 크기
)

plt.title("Ligand distribution PCA: PC1 vs PC2")  # 그래프 제목
plt.xlabel("PC1")                               # x축 레이블
plt.ylabel("PC2")                               # y축 레이블
plt.legend(title="Ligands", bbox_to_anchor=(1.05, 1), loc='upper left')  # 범례 위치 조정
plt.tight_layout()
plt.show()


PCA 분석이 Pd(I)를 유도하는 리간드와 그렇지 않은 리간드를 구분하지 못한다는 점을 확인해봅시다. 이를 위해 이전과 동일한 방식으로 전체 데이터를 시각화하되, 목표 리간드(즉, Pd(I) 이합체 생성을 유도하는 리간드)에 해당하는 데이터 포인트를 강조하여 표시합니다.


In [ ]:
pos_refs = [16, 41, 54, 113]   # Pd(I) 유도 리간드
neg_refs = [21]                # Pd(I) 유도하지 않는 리간드

# PCA 시각화: 전체 리간드 + 양성/음성 강조
plt.figure(figsize=(8, 6))

# 전체 데이터 포인트 플롯 (연한 회색)
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=ligands,             # 데이터프레임 사용
    x="PC1",             # x축: 첫 번째 주성분
    y="PC2",             # y축: 두 번째 주성분
    hue="Type",          # 색상: 리간드 유형
    palette="Set2",      # 색상 팔레트
    s=60                 # 점 크기
)

# 양성 리간드 강조 (파란색 별표)
plt.scatter(
    ligands.loc[pos_refs, "PC1"],
    ligands.loc[pos_refs, "PC2"],
    color="blue",
    marker="*",
    s=200,
    label="Pd(I) positive"
)

# 음성 리간드 강조 (빨간색 X 표시)
plt.scatter(
    ligands.loc[neg_refs, "PC1"],
    ligands.loc[neg_refs, "PC2"],
    color="red",
    marker="X",
    s=100,
    label="Pd(I) negative"
)

# 그래프 제목 및 축 설정
plt.title("PCA visualization")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend(loc='best')
plt.tight_layout()
plt.show()

PCA 결과만으로는 뚜렷한 정보를 얻기 어렵다는 점을 명확히 확인할 수 있습니다. Pd(I)를 유도하는 리간드 중 두 개는 다른 유도 리간드보다 오히려 Pd(I)를 유도하지 않는 리간드와 더 가깝게 위치하고 있습니다. 이는 PCA 분석만으로는 새로운 Pd(I) 종을 제안하기에 충분하지 않다는 것을 보여줍니다.


# 2. 일반 특성 기반 클러스터링

이제 접근 방식을 바꿔보겠습니다! 먼저, k-means 알고리즘을 사용하여 리간드들을 효율적으로 군집화할 수 있는지 확인해봅니다. 이 과정을 통해 실험적으로 선별해야 할 후보 분자의 수를 줄일 수 있습니다.

k 값을 달리하여 k-means 클러스터링을 수행하고, inertia, silhouette score를 기준으로 최적의 클러스터 개수를 선택합니다.

아래는 사용할 특성 컬럼이며, 먼저 데이터를 표준화(정규화)해야 합니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 클러스터링에 사용할 특성 컬럼 정의
process = ['E(HOMO)', 'E(LUMO)', 'He8_steric', 'PA', 'Q(B)', 'BE(B)', 'P-B', 'DP-A(B)', 'DA-P-A(B)', 'Q(Au)', 'BE(Au)',
           'Au-Cl', 'Au-P', 'DP-A(Au)', 'DA-P-A(Au)', 'Q(Pd)', 'BE(Pd)', 'Pd-Cl trans', 'P-Pd', 'DP-A(Pd)',
           'DA-P-A(Pd)', 'Q(Pt)', 'BE(Pt)', 'P-Pt', 'DP-A(Pt)', 'DA-P-A(Pt)', '<(H3P)Pt(PH3)', "S4'"]
drop = ['Type', "PC1", "PC2", "PC3", "PC4"]

# 선택한 특성들만 추출
X = ligands[process].copy()

# 특성 표준화 (평균 0, 표준편차 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 클러스터 수에 따라 inertia와 silhouette score 계산
inertias = []
silhouettes = []

K_range = range(2, 15)  # 클러스터 수 k = 2 ~ 14

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)

    inertias.append(kmeans.inertia_)  # 관성 값 저장
    silhouettes.append(silhouette_score(X_scaled, kmeans.labels_))  # 실루엣 점수 저장

# 결과 시각화
plt.figure(figsize=(12, 5))

# Inertia 시각화
plt.subplot(1, 2, 1)
plt.plot(K_range, inertias, marker='o')
plt.title('Inertia vs Number of Clusters')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')

# Silhouette score 시각화
plt.subplot(1, 2, 2)
plt.plot(K_range, silhouettes, marker='o', color='green')
plt.title('Silhouette Score vs Number of Clusters')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()


Inertia에서는 명확한 elbow 지점이 보이지 않아 최적의 클러스터 수(k)를 결정하기 어렵습니다.

반면 Silhouette Score는 k=7 부근에서 로컬 최대값을 가지므로, 이를 기반으로 볼 때 **k=7이 하나의 유력한 후보**가 될 수 있습니다.


## Optional

원 논문에서는 클러스터 수를 8로 설정하였습니다. 이는 k=6~8 구간에서 유사한 성능을 보였고, 원래 리간드 분류(class)의 개수도 8이었기 때문입니다.

이번 과제에서는 **k=8로 K-mean 모델을 구성하고, 1000개의 서로 다른 random seed를 사용하여 모델을 반복 학습**합니다. 그 후, **각 리간드가 Pd(I)를 유도하는 리간드들과 동일한 클러스터에 속한 횟수**를 집계해봅니다.


In [ ]:
import numpy as np
from sklearn.cluster import KMeans

# Pd(I)를 유도하는 리간드 인덱스
pos_refs = [16, 41, 54, 113]

# 클러스터링에 사용할 특성 컬럼 정의 및 데이터 준비
X = ligands[process].copy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 클러스터 수 설정
k = 8
n_iter = 1000

# 각 리간드가 양성 리간드와 같은 클러스터에 속한 횟수 저장용 배열
counts = np.zeros(len(X_scaled), dtype=int)

# 1000번 서로 다른 seed로 클러스터링 수행
for seed in range(n_iter):
    kmeans = KMeans(n_clusters=k, random_state=seed)
    labels = kmeans.fit_predict(X_scaled)

    # 양성 리간드들이 속한 클러스터들 집합
    pos_clusters = set(labels[i] for i in pos_refs)

    # 해당 클러스터에 속한 리간드마다 카운트 증가
    for i, label in enumerate(labels):
        if label in pos_clusters:
            counts[i] += 1

# 결과를 데이터프레임에 저장
ligands['Pos_Cluster_Count'] = counts

# 가장 자주 양성 리간드와 같은 클러스터에 들어간 상위 몇 개 확인
ligands[['Pos_Cluster_Count']].sort_values(by='Pos_Cluster_Count', ascending=False).head(10)


이 과정을 통해 각 리간드가 양성 리간드(Pd(I) 유도 리간드)와 동일한 클러스터에 포함된 빈도를 기반으로 한 통계 데이터프레임을 얻을 수 있습니다.

이제 해당 값을 정규화하여 **Score**로 정의하고, 이 점수를 기준으로 필터링을 수행합니다. 구체적으로는, **전체 1000회 중 75% 이상(즉, 750회 이상) 양성 리간드와 같은 클러스터에 속한 리간드들만 추출**합니다.


In [ ]:
# 총 반복 횟수로 정규화하여 Score 컬럼 추가 (0~1 사이 값)
ligands['Score'] = ligands['Pos_Cluster_Count'] / n_iter

# Score가 0.75 이상인 리간드만 필터링
high_score_ligands = ligands[ligands['Score'] >= 0.75]

# 결과 확인
high_score_ligands[['Score']].sort_values(by='Score', ascending=False)


선택된 리간드들의 인덱스를 시각화하여 어떤 리간드들이 필터링되었는지 확인하고, **Pd(I)를 유도하지 않는 리간드(인덱스 21번)** 이 포함되지 않았는지 검증합니다.  

이 인덱스 배열은 나중에 원본 데이터에서 리간드를 선택하거나 필터링하는 데 사용할 수 있습니다.

In [ ]:
# 선택된 리간드들의 인덱스 추출
selected_indices = high_score_ligands.index.to_list()

# 인덱스 출력
print("Selected ligand indices (Score ≥ 0.75):")
print(selected_indices)

# 비유도 리간드가 포함되어 있는지 확인
if 21 in selected_indices:
    print("⚠️ Warning: Ligand #21 (non-inducing) is included!")
else:
    print("✅ Ligand #21 (non-inducing) is NOT included in the selected group.")


선택된 리간드 그룹이 **여러 가지 리간드 유형(Type)** 을 포함하고 있는지도 확인합니다.  
이는 클러스터링이 단순히 분류(class) 기준이 아닌, 다양한 특성 정보를 기반으로 의미 있는 분류를 수행했음을 보여줍니다.

In [ ]:
# 선택된 리간드들의 유형 확인
selected_types = high_score_ligands['Type'].value_counts()

# 결과 출력
print("Distribution of ligand types in the selected group:")
print(selected_types)

# 시각화 (막대 그래프)
import seaborn as sns

plt.figure(figsize=(6, 4))
sns.countplot(data=high_score_ligands, x='Type', order=selected_types.index)
plt.title('Ligand Type Distribution in Selected Group')
plt.xlabel('Ligand Type')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

우리는 이제 Pd(I)를 유도하지 않는 문제의 리간드(21번)를 성공적으로 제거하였습니다.  
또한, 선택된 클러스터에는 다양한 유형(Type)의 리간드들이 포함되어 있어,  
클러스터링이 단순한 분류 이상의 통찰을 제공하고 있음을 보여줍니다.

이는 새로운 Pd(I) 이합체 후보를 발굴하기 위한 **좋은 첫 단계**입니다.


# 3. DFT 기반 특성 (DFT-specific descriptors)

첫 번째 클러스터링을 통해 후보 리간드 공간을 약 28.7%로 축소하였으며, 전체 348개 중 98개의 리간드를 유지할 수 있었습니다.  

이후 일반적인 분자 특성을 사용하여 k-mean 클러스터링을 다시 수행했지만, 초기 필터링을 더 세분화하는 데에는 실패했습니다.  

따라서 이 문제에 특화된 특성이 필요하게 되었고, **Pd(I) 아이오딘 브릿지와 관련된 DFT 기반 특성** 42개가 새롭게 계산되어 데이터셋에 포함되었습니다.

이번에는 DFT 계산을 직접 수행하지 않고, **P-C 결합을 포함하는 리간드에 대해 이미 계산된 결과**를 불러옵니다.  
이 데이터프레임에는 총 66개의 리간드가 포함되어 있습니다.


In [44]:
# DFT 기반 특성을 포함한 데이터프레임 불러오기 및 시각화 (파일 경로: data/Dimer_LKB_P.csv)


#Solution
# %load https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/notebooks/week5/solution_06.py

,No.,Type,E(HOMO),E(LUMO),He8_steric,PA,Q(Pd),BE(Pd),Pd-Cl trans,P-Pd,...,G_NBO_Pd,G_NBO_P,G_LENGTH_Pd_P,G_LENGTH_Pd_Pd,G_TORSION_Pd_I_I_Pd,G_ORDER_Pd_Pd,PC1',PC2',PC3',PC4'
0,15,R,-0.17500,0.02350,37.82,247.52,-1.257,30.03,2.379,2.337,...,-0.379685,1.297005,2.312152,3.226557,103.059842,0.1171,-0.986788,0.414056,0.017615,-0.632639
1,16,R,-0.17267,0.02754,23.38,250.80,-1.200,23.90,2.382,2.418,...,-0.278010,1.232615,2.413734,4.305073,174.451579,0.0264,1.198717,1.162347,0.901051,-0.055612
2,19,R,-0.18340,0.01900,31.31,243.63,-1.282,28.25,2.377,2.308,...,-0.340260,1.205245,2.314010,3.620629,124.149654,0.0713,-0.146169,-0.318877,0.877642,-0.227544
3,36,R,-0.17840,0.02300,19.69,245.86,-1.275,34.79,2.379,2.321,...,-0.365385,1.280040,2.317294,3.187955,101.341176,0.1254,-1.135555,0.275201,0.181931,-0.516078
4,37,R,-0.17790,0.02220,18.53,246.58,-1.253,35.72,2.380,2.334,...,-0.387150,1.284050,2.346424,3.102411,95.391197,0.1369,-0.904739,0.447820,0.106396,-0.301205


# 4. 최종 클러스터링: 새로운 리간드 후보 탐색

새로운 DFT descriptors이 계산되었으므로, 이를 활용하여 두 번째 k-Means 클러스터링을 수행하고, 후보 리간드 공간을 한층 더 줄일 수 있는지 확인해봅니다.  

이전과 동일한 방식으로 클러스터 수 k를 변화시키며 최적값을 선택합니다.

다음은 클러스터링에서 제외해야 할 컬럼이며, positive 리간드의 참조 인덱스도 새로운 데이터프레임 기준으로 수정되었습니다.


In [ ]:
# 클러스터링에서 제외할 컬럼
drop = ["Type", "PC1", "PC2", "PC3", "PC4", "PC1'", "PC2'", "PC3'", "PC4'"]

# 양성 리간드 인덱스 (새로운 DFT 데이터 기준)
refs = [1, 8, 18, 36]

# 사용할 특성만 선택
X_dft = df_dft.drop(columns=drop)
scaler = StandardScaler()
X_scaled_dft = scaler.fit_transform(X_dft)

# 클러스터 수별 평가 지표 저장용 리스트
inertias = []
silhouettes = []

K_range = range(2, 15)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled_dft)

    inertia = kmeans.inertia_
    inertias.append(inertia)
    silhouettes.append(silhouette_score(X_scaled_dft, kmeans.labels_))

# 세 지표 시각화 (3개의 subplot)
plt.figure(figsize=(10, 4))

# Inertia
plt.subplot(1, 2, 1)
plt.plot(K_range, inertias, marker='o')
plt.title('Inertia vs Number of Clusters')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')

# Silhouette Score
plt.subplot(1, 2, 2)
plt.plot(K_range, silhouettes, marker='^', color='green')
plt.title('Silhouette Score vs Number of Clusters')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()

이번 경우에도 Inertia와 Silhouette Score 모두  클러스터 수(k)에 대한 명확한 최적값을 제시하지 못하는 것으로 보입니다.  
즉, **이번 클러스터링에서도 지표들이 그다지 유의미한 정보를 제공하지 않습니다.**


## 선택 과제 (Optional)

원 논문에서는 클러스터 수를 6으로 설정하였습니다.  
이번에는 **k=6으로 K-평균 모델을 구성하고, 1000개의 서로 다른 random seed를 사용하여 모델을 반복 학습**합니다.  
그 후, **각 리간드가 양성 리간드들과 동일한 클러스터에 속한 횟수**를 집계해봅니다.


In [ ]:
# 양성 리간드 인덱스 (DFT 기반 데이터프레임 기준)
refs = [1, 8, 18, 36]

# 클러스터 수 및 반복 횟수 설정
k = 6
n_iter = 1000

# 카운트 저장용 배열 초기화
counts = np.zeros(len(X_scaled_dft), dtype=int)

# 1000번 서로 다른 시드로 클러스터링 수행
for seed in range(n_iter):
    kmeans = KMeans(n_clusters=k, random_state=seed)
    labels = kmeans.fit_predict(X_scaled_dft)

    # 양성 리간드들이 속한 클러스터 번호 집합
    pos_clusters = set(labels[i] for i in refs)

    # 해당 클러스터에 속하는 리간드 카운트 증가
    for i, label in enumerate(labels):
        if label in pos_clusters:
            counts[i] += 1

# 결과를 데이터프레임에 저장
df_dft['Pos_Cluster_Count'] = counts

# 상위 리간드 확인 (양성 리간드와 자주 같은 클러스터에 속한 순)
df_dft[['Pos_Cluster_Count']].sort_values(by='Pos_Cluster_Count', ascending=False).head(10)


이제 전체 1000회 클러스터링 중에서, 양성 리간드들과 **50% 이상(즉, 500회 이상)** 동일한 클러스터에 속한 리간드들을 선택합니다.  
이러한 리간드들은 Pd(I) 이합체 형성 가능성이 높은 유망 후보로 간주될 수 있습니다.


In [ ]:
# 50% 이상 양성 리간드와 같은 클러스터에 속한 리간드 선택
df_dft['Score'] = df_dft['Pos_Cluster_Count'] / n_iter

# 0.5 이상인 리간드만 필터링
selected_dft_ligands = df_dft[df_dft['Score'] >= 0.5]

# 결과 확인
selected_dft_ligands[['Score']].sort_values(by='Score', ascending=False)


# 5. 실험적 검증 (Experimental validation)

두 번째 클러스터링 결과, 기존에 알려진 4개의 Pd(I) 이합체 유도 리간드와 함께 **총 21개의 새로운 리간드**가 동일한 클러스터에 포함되었습니다.  

이 예측 결과를 바탕으로 실험을 진행한 결과는 다음과 같습니다:

<div align="left">
<img src="https://raw.githubusercontent.com/jfjoung/AI_For_Chemistry/main/img/exp_validation.png" width="900"/>
</div>

놀랍게도 **8개의 새로운 Pd(I) 이합체가 발견**되었습니다!  
이러한 성과는 **데이터가 매우 적고, 리간드-금속 분포(speciation) 관계에 대한 이해도 부족한 상황**에서 얻어진 것입니다.

해당 리간드들의 구조는 원 논문의 SI를 참고하면 확인할 수 있으며, 그 구조들이 서로 매우 이질적이라는 점도 알 수 있습니다.  

아마도 사람 연구자라면 직관적으로 제외했을 분자들이지만, ML은 이들 사이의 **비직관적인 유사성**을 발견하고, 의미 있는 실험적 제안을 제공할 수 있었습니다.


# 결론 (Conclusion)

이번 노트북에서는 k-means처럼 단순한 알고리즘도 **적절하게 활용하면 매우 강력한 도구가 될 수 있음**을 보여주었습니다.  
기계학습과 화학의 경계에서 활동하는 연구자들은 **두 분야의 지식을 결합하여 놀라운 성과**를 이끌어낼 수 있습니다.  
이 예제가 여러분에게 지속적인 학습의 동기를 부여하길 바랍니다!

마지막으로, 원 논문의 전체 코드가 포함된 GitHub 저장소는 [여기](https://github.com/J-Hueffel/PdDimer)에서 확인하실 수 있습니다.  
또한, 이 훌륭한 연구를 진행한 [Schoenebeck 그룹](https://www.schoenebeck.oc.rwth-aachen.de/)에 감사의 인사를 전합니다.